# Azure DevOps

In [ ]:
CREATE OR REPLACE FUNCTION get_builds(PROJECT_NAME STRING, REPO_ID STRING)
  RETURNS VARIANT
  LANGUAGE PYTHON
  RUNTIME_VERSION = 3.9
  HANDLER = 'get_builds_handler'
  EXTERNAL_ACCESS_INTEGRATIONS = (azure_devops_integration)
  PACKAGES = ('requests')
  SECRETS = ('pat' = PROD.RAW.ADO_PAT)
AS
$$
import requests
from datetime import datetime, timedelta
import _snowflake
from urllib.parse import urlencode
from requests.auth import HTTPBasicAuth

def get_builds_handler(PROJECT_NAME, REPO_ID):
    # Define the variables
    api_version = "7.1"
    
    pat = _snowflake.get_generic_secret_string('pat')


    yesterday = datetime.now() - timedelta(days=1)
    yesterday_year = yesterday.year
    yesterday_month = yesterday.month
    yesterday_day_no = yesterday.day
    
    min_time = datetime(yesterday_year, yesterday_month, yesterday_day_no, 0, 0, 0).strftime("%Y-%m-%dT%H:%M:%SZ")  # Start date
    max_time = datetime(yesterday_year, yesterday_month, yesterday_day_no, 23, 59, 59).strftime("%Y-%m-%dT%H:%M:%SZ")  # End date



    # Base URL
    base_url = f"https://dev.azure.com/phlexglobal/{PROJECT_NAME}/_apis/build/builds"
    params = {
        "repositoryId": REPO_ID,
        "minTime": min_time,
        "maxTime": max_time,
        "repositoryType": "TfsGit",
        "api-version": api_version
    }
    url = f"{base_url}?{urlencode(params)}"

    # Headers
    headers = {
        "Accept": "application/json"
    }
    
    # Make GET request
    response = requests.get(
        url,
        headers=headers,
        auth=HTTPBasicAuth("", pat)
    )

    try:
        parsed_data = response.json()
        value = parsed_data.get("value", [])
        count = parsed_data.get("count", 0)
        builds = []
    
        for build in value:
            parsed_build = {
                "URI": build.get("uri"),
                "URL": build.get("url"),
                "LOGS": build.get("logs"),
                "TAGS": build.get("tags", []),
                "PLANS": build.get("plans", []),
                "QUEUE": build.get("queue"),
                "_LINKS": build.get("_links"),
                "REASON": build.get("reason"),
                "RESULT": build.get("result"),
                "STATUS": build.get("status"),
                "PROJECT": build.get("project"),
                "PRIORITY": build.get("priority"),
                "QUEUETIME": build.get("queueTime"),
                "STARTTIME": build.get("startTime"),
                "DEFINITION": build.get("definition"),
                "FINISHTIME": build.get("finishTime"),
                "PARAMETERS": None,  # This field is missing in JSON; set to None
                "PROPERTIES": build.get("properties"),
                "REPOSITORY": build.get("repository"),
                "BUILDNUMBER": build.get("buildNumber"),
                "REQUESTEDBY": build.get("requestedBy"),
                "TRIGGERINFO": build.get("triggerInfo"),
                "REQUESTEDFOR": build.get("requestedFor"),
                "SOURCEBRANCH": build.get("sourceBranch"),
                "LASTCHANGEDBY": build.get("lastChangedBy"),
                "SOURCEVERSION": build.get("sourceVersion"),
                "LASTCHANGEDDATE": build.get("lastChangedDate"),
                "ORCHESTRATIONPLAN": build.get("orchestrationPlan"),
                "RETAINEDBYRELEASE": build.get("retainedByRelease"),
                "VALIDATIONRESULTS": build.get("validationResults", []),
                "TEMPLATEPARAMETERS": build.get("templateParameters"),
                "BUILDNUMBERREVISION": build.get("buildNumberRevision"),
                "APPENDCOMMITMESSAGETORUNNAME": build.get("appendCommitMessageToRunName"),
            }
            
            builds.append(parsed_build)
        
        return {"count": count, "builds": builds}  # Fix the unhashable type issue

    except Exception as e:
        return {"error": str(e)}
$$;

In [ ]:
CREATE OR REPLACE FUNCTION get_pull_request(PROJECT_NAME STRING, REPO_ID STRING)
  RETURNS VARIANT
  LANGUAGE PYTHON
  RUNTIME_VERSION = 3.9
  HANDLER = 'get_pull_request_handler'
  EXTERNAL_ACCESS_INTEGRATIONS = (azure_devops_integration)
  PACKAGES = ('requests')
  SECRETS = ('pat' = PROD.RAW.ADO_PAT)
AS
$$
import requests
from datetime import datetime, timedelta
import _snowflake
from urllib.parse import urlencode
from requests.auth import HTTPBasicAuth



def get_pull_request_handler(PROJECT_NAME, REPO_ID):
    # Define the variables
    api_version = "7.1"
    pat = _snowflake.get_generic_secret_string('pat')
    # Get yesterday's date
    yesterday = datetime.utcnow() - timedelta(days=14)
    min_time = yesterday.replace(hour=0, minute=0, second=0).strftime("%Y-%m-%dT%H:%M:%SZ")
    max_time = yesterday.replace(hour=23, minute=59, second=59).strftime("%Y-%m-%dT%H:%M:%SZ")
    year_2024 = datetime(2024, 1, 1).strftime("%Y-%m-%dT%H:%M:%SZ")
    utc_now = datetime.utcnow()
    utc_now_min = utc_now - timedelta(days=14)
    
    base_url = f"https://dev.azure.com/phlexglobal/{PROJECT_NAME}/_apis/git/repositories/{REPO_ID}/pullrequests?searchCriteria.status=all&searchCriteria.minTime={year_2024}&searchCriteria.maxTime={utc_now}&searchCriteria.timeRangeType=created&api-version=7.1"

    # Retrieve the PAT securely from Snowflake secrets
    # pat = _snowflake.get_generic_secret_string('pat')
    
    # Headers
    headers = {
        "Accept": "application/json"
    }
    
    # Make GET request
    response = requests.get(
        base_url,
        headers=headers,
        auth=HTTPBasicAuth("", pat)
    )

    try:
        parsed_data = response.json()
        count = parsed_data.get("count", 0)
        value = parsed_data.get("value", [])
        
        flattened_prs = []
        for pr in value:
            flat_pr = {
                'repository': pr.get('repository', {}),
                'pullRequestId': pr.get('pullRequestId'),
                'codeReviewId': pr.get('codeReviewId'),
                'status': pr.get('status'),
                'createdBy': pr.get('createdBy', {}),
                'creationDate': pr.get('creationDate'),
                'closedDate': pr.get('closedDate'),
                'title': pr.get('title'),
                'description': pr.get('description'),
                'sourceRefName': pr.get('sourceRefName'),
                'targetRefName': pr.get('targetRefName'),
                'mergeStatus': pr.get('mergeStatus'),
                'isDraft': pr.get('isDraft'),
                'mergeId': pr.get('mergeId'),
                'lastMergeSourceCommit': pr.get('lastMergeSourceCommit', {}),
                'lastMergeTargetCommit': pr.get('lastMergeTargetCommit', {}),
                'lastMergeCommit': pr.get('lastMergeCommit', {}),
                'reviewers': pr.get('reviewers', []),
                'url': pr.get('url'),
                'completionOptions': pr.get('completionOptions', {}),
                'supportsIterations': pr.get('supportsIterations')
            }
            flattened_prs.append(flat_pr)
        
        return {"count": count, "pullRequests": flattened_prs}

    except Exception as e:
        return {"error": str(e)}
$$;

In [ ]:
CREATE OR REPLACE FUNCTION get_branches(project STRING, repo STRING)
  RETURNS ARRAY
  LANGUAGE PYTHON
  RUNTIME_VERSION = 3.9
  HANDLER = 'get_branches_handler'
  EXTERNAL_ACCESS_INTEGRATIONS = (azure_devops_integration)
  PACKAGES = ('requests')
  SECRETS = ('pat' = prod.raw.ado_pat)
AS
$$
import requests
from requests.auth import HTTPBasicAuth
import _snowflake

def get_branches_handler(project, repo):
    # API URL
    url = f"https://dev.azure.com/phlexglobal/{project}/_apis/sourceProviders/TfsGit/branches?repository={repo}&api-version=7.1"
    # Retrieve PAT securely from Snowflake secrets
    pat = _snowflake.get_generic_secret_string('pat')

    # Headers
    headers = {
        "Accept": "application/json"
    }

    # Make the GET request
    response = requests.get(url, headers=headers, auth=HTTPBasicAuth("", pat))

    # Handle response
    if response.status_code == 200:
        branches = response.json()

        # Ensure 'value' exists and is a list
        if 'value' in branches and isinstance(branches['value'], list):
            # Extract branch names and remove 'refs/heads/' prefix
            branches_name = [i.replace('refs/heads/', '') for i in branches['value']]
            return branches_name
        else:
            # Handle unexpected API structure
            return [f"Unexpected response format: {branches}"]
    else:
        # Handle API errors
        return [f"Error: {response.status_code}, {response.text}"]
$$;

In [ ]:
CREATE OR REPLACE FUNCTION get_commits(repo STRING, branch STRING)
  RETURNS VARIANT
  LANGUAGE PYTHON
  RUNTIME_VERSION = 3.9
  HANDLER = 'get_commits_handler'
  EXTERNAL_ACCESS_INTEGRATIONS = (azure_devops_integration)
  PACKAGES = ('requests')
  SECRETS = ('pat' = prod.raw.ado_pat)
AS
$$
import requests
from requests.auth import HTTPBasicAuth
import time
import _snowflake

def get_commits_handler(repo, branch):
    # API URL
    url = f"https://dev.azure.com/phlexglobal/_apis/git/repositories/{repo}/commits?searchCriteria.itemVersion.version={branch}&api-version=7.1"

    pat = _snowflake.get_generic_secret_string('pat')
    # Headers
    headers = {
        "Accept": "application/json"
    }

    # Retry logic with exponential backoff
    retries = 3
    backoff_factor = 2
    for attempt in range(retries):
        try:
            # Make the GET request
            response = requests.get(url, headers=headers, auth=HTTPBasicAuth("", pat))
            response.raise_for_status()  # Raise an exception for HTTP errors
            
            # Parse response JSON
            commits = response.json()

            # Extract commit count and metadata
            if response.status_code == 200:
                return commits
            else:
                # Handle unexpected API structure
                return {"error": "Unexpected API response", "response": commits}
        except requests.exceptions.RequestException as e:
            if attempt < retries - 1:
                time.sleep(backoff_factor ** attempt)  # Exponential backoff
            else:
                # Return error after exhausting retries
                return {"error": "Request failed", "message": str(e)}
$$;

In [ ]:
CREATE OR REPLACE FUNCTION PROD.RAW.ado_secrets()
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'ado_secrets'
EXTERNAL_ACCESS_INTEGRATIONS = (AZURE_DEVOPS_INTEGRATION)
SECRETS = ('pat'=PROD.RAW.ADO_PAT)
AS '
import _snowflake

def ado_secrets():   
  my_api_key = _snowflake.get_generic_secret_string(''pat'') 
  return my_api_key
';

# Jira

In [ ]:
CREATE OR REPLACE FUNCTION PROD.RAW.jira_username()
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'jira_username'
EXTERNAL_ACCESS_INTEGRATIONS = (jira_integration)
SECRETS = ('jira_username'=PROD.RAW.jira_username)
AS '
import _snowflake

def jira_username():   
  my_api_key = _snowflake.get_generic_secret_string(''jira_username'') 
  return my_api_key
';

In [ ]:
CREATE OR REPLACE FUNCTION PROD.RAW.jira_api_token()
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'jira_api_token'
EXTERNAL_ACCESS_INTEGRATIONS = (jira_integration)
SECRETS = ('jira_api_token'=PROD.RAW.jira_api_token)
AS '
import _snowflake

def jira_api_token():   
  my_api_key = _snowflake.get_generic_secret_string(''jira_api_token'') 
  return my_api_key
';

In [ ]:
CREATE OR REPLACE FUNCTION get_sprint_issues(
    sprint_id int
)
RETURNS VARIANT
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
EXTERNAL_ACCESS_INTEGRATIONS = (jira_integration)
PACKAGES = ('requests', 'pandas')
HANDLER = 'get_sprint_issues'
SECRETS = ('jira_username' = PROD.RAW.jira_username, 'jira_api_token'= PROD.RAW.jira_api_token)
AS 
$$
import requests
import pandas as pd
import time
import _snowflake

def get_sprint_issues(sprint_id):
    jira_username = _snowflake.get_generic_secret_string('jira_username')
    jira_api_token = _snowflake.get_generic_secret_string('jira_api_token')
    if not jira_username or not jira_api_token:
        return {"status": "error", "message": "Missing credentials"} 
    time.sleep(3)
    params = {
        "startAt": 0,
        "maxResults": 1000
    }

    url = f"https://phlexglobal.atlassian.net/rest/agile/1.0/sprint/{sprint_id}/issue"
    headers = {"Accept": "application/json"}

    for attempt in range(3):
        response = requests.get(
            url,
            auth=(jira_username, jira_api_token),
            headers=headers,
            params=params
        )

        if response.status_code == 429:  # rate limited
            retry_after = int(response.headers.get("Retry-After", "3"))
            time.sleep(retry_after)
            continue
        elif response.status_code != 200:
            return {
                "status": "error",
                "code": response.status_code,
                "message": response.text
            }
        else:
            break

    issues = response.json().get("issues", [])

    needed_columns = [
        "id", "key", "self", "expand",
        "fields.created", "fields.updated", 
        "fields.customfield_10006", 
        "fields.customfield_11302", 
        "fields.status.description", 
        "fields.status.iconUrl", 
        "fields.status.id", 
        "fields.status.name", 
        "fields.status.self", 
        "fields.status.statusCategory.colorName",
        "fields.status.statusCategory.id",
        "fields.status.statusCategory.key",
        "fields.status.statusCategory.name",
        "fields.status.statusCategory.self"
    ]

    try:
        df = pd.json_normalize(issues)
        df = df[needed_columns].reset_index()
    except KeyError:
        return []

    return df.to_dict(orient="records")
    print(f"sprint_id : {sprint_id}")

$$;


# Mend 

In [ ]:
-- mend jwt token
CREATE OR REPLACE SECRET mend_orgUuid_jwtToken
TYPE = GENERIC_STRING
seCRET_STRING = '71003aaf-b8e7-4e7b-a622-736b775a0eff'
COMMENT = 'mend orgUuid to get jwtToken';

CREATE OR REPLACE SECRET mend_user_key_jwtToken
TYPE = GENERIC_STRING
SECRET_STRING = '8ded55a66bb04810bc4ce5d227cff997dd178fe0b56b4495b77b16417ab1032b'
COMMENT = 'mend userKey to get jwtToken';

CREATE OR REPLACE SECRET mend_email_jwtToken
TYPE = GENERIC_STRING
SECRET_STRING = 'ajoyce@phlexglobal.com'
COMMENT = 'mend email to get jwtToken';

In [ ]:
CREATE OR REPLACE FUNCTION PROD.RAW.mend_orguuid_jwttoken()
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'mend_orguuid_jwttoken'
EXTERNAL_ACCESS_INTEGRATIONS = (mend_api_integration)
SECRETS = ('mend_orguuid_jwttoken'=PROD.RAW.mend_orgUuid_jwtToken)
AS '
import _snowflake

def mend_orguuid_jwttoken():   
  my_api_key = _snowflake.get_generic_secret_string(''mend_orguuid_jwttoken'') 
  return my_api_key
';

In [ ]:
CREATE OR REPLACE FUNCTION PROD.RAW.mend_user_key_jwttoken()
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'mend_user_key_jwttoken'
EXTERNAL_ACCESS_INTEGRATIONS = (mend_api_integration)
SECRETS = ('mend_user_key_jwttoken'=PROD.RAW.mend_user_key_jwttoken)
AS '
import _snowflake

def mend_user_key_jwttoken():   
  my_api_key = _snowflake.get_generic_secret_string(''mend_user_key_jwttoken'') 
  return my_api_key
';

In [ ]:
CREATE OR REPLACE FUNCTION PROD.RAW.mend_email_jwttoken()
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'mend_email_jwttoken'
EXTERNAL_ACCESS_INTEGRATIONS = (mend_api_integration)
SECRETS = ('mend_email_jwttoken'=PROD.RAW.mend_email_jwttoken)
AS '
import _snowflake

def mend_email_jwttoken():   
  my_api_key = _snowflake.get_generic_secret_string(''mend_email_jwttoken'') 
  return my_api_key
';

# Sonarcloud

In [ ]:
CREATE OR REPLACE FUNCTION PROD.RAW.sonarcloud_api_key()
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'sonarcloud_api_key'
EXTERNAL_ACCESS_INTEGRATIONS = (sonarcloud_api_integration)
SECRETS = ('sonarcloud_api_key'=PROD.RAW.sonarcloud_api_key)
AS '
import _snowflake

def sonarcloud_api_key():   
  my_api_key = _snowflake.get_generic_secret_string(''sonarcloud_api_key'') 
  return my_api_key
';

# Defender

In [ ]:
CREATE OR REPLACE FUNCTION PROD.RAW.ADO_DEFENDER_TENANT_ID()
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'ADO_DEFENDER_TENANT_ID'
EXTERNAL_ACCESS_INTEGRATIONS = (microsoft_api_integration)
SECRETS = ('ADO_DEFENDER_TENANT_ID'=PROD.RAW.ADO_DEFENDER_TENANT_ID)
AS '
import _snowflake

def ADO_DEFENDER_TENANT_ID():   
  my_api_key = _snowflake.get_generic_secret_string(''ADO_DEFENDER_TENANT_ID'') 
  return my_api_key
';

CREATE OR REPLACE FUNCTION PROD.RAW.ADO_DEFENDER_CLIENT_ID()
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'ADO_DEFENDER_CLIENT_ID'
EXTERNAL_ACCESS_INTEGRATIONS = (microsoft_api_integration)
SECRETS = ('ADO_DEFENDER_CLIENT_ID'=PROD.RAW.ADO_DEFENDER_CLIENT_ID)
AS '
import _snowflake

def ADO_DEFENDER_CLIENT_ID():   
  my_api_key = _snowflake.get_generic_secret_string(''ADO_DEFENDER_CLIENT_ID'') 
  return my_api_key
';

CREATE OR REPLACE FUNCTION PROD.RAW.ADO_DEFENDER_CLIENT_SECRET()
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'ADO_DEFENDER_CLIENT_SECRET'
EXTERNAL_ACCESS_INTEGRATIONS = (microsoft_api_integration)
SECRETS = ('ADO_DEFENDER_CLIENT_SECRET'=PROD.RAW.ADO_DEFENDER_CLIENT_SECRET)
AS '
import _snowflake

def ADO_DEFENDER_CLIENT_SECRET():   
  my_api_key = _snowflake.get_generic_secret_string(''ADO_DEFENDER_CLIENT_SECRET'') 
  return my_api_key
';

CREATE OR REPLACE FUNCTION PROD.RAW.ADO_DEFENDER_SUBSCRIPTION_ID()
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'ADO_DEFENDER_SUBSCRIPTION_ID'
EXTERNAL_ACCESS_INTEGRATIONS = (microsoft_api_integration)
SECRETS = ('ADO_DEFENDER_SUBSCRIPTION_ID'=PROD.RAW.ADO_DEFENDER_SUBSCRIPTION_ID)
AS '
import _snowflake

def ADO_DEFENDER_SUBSCRIPTION_ID():   
  my_api_key = _snowflake.get_generic_secret_string(''ADO_DEFENDER_SUBSCRIPTION_ID'') 
  return my_api_key
';

In [ ]:
CREATE OR REPLACE FUNCTION PROD.RAW.new_relic_secrets()
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'ado_secrets'
EXTERNAL_ACCESS_INTEGRATIONS = (newrelic_api_integration)
SECRETS = ('nr'=PROD.RAW.NEW_RELIC)
AS '
import _snowflake

def ado_secrets():   
  my_api_key = _snowflake.get_generic_secret_string(''nr'') 
  return my_api_key
';

Checkmarx


In [ ]:
CREATE OR REPLACE FUNCTION PROD.RAW.checkmarx_secret()
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'checkmarx_secret'
EXTERNAL_ACCESS_INTEGRATIONS = (checkmarx_api_integration)
SECRETS = ('checkmarx'=PROD.RAW.checkmarx)
AS '
import _snowflake

def checkmarx_secret():   
  my_api_key = _snowflake.get_generic_secret_string(''checkmarx'') 
  return my_api_key
';

In [ ]:
CREATE OR REPLACE FUNCTION PROD.RAW.pagerduty_api_key()
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'pagerduty_secret'
EXTERNAL_ACCESS_INTEGRATIONS = (pagerduty_api_integration)
SECRETS = ('pagerduty'=PROD.RAW.pagerduty_api_key)
AS '
import _snowflake

def pagerduty_secret():   
  my_api_key = _snowflake.get_generic_secret_string(''pagerduty'') 
  return my_api_key
';

In [ ]:

CREATE OR REPLACE FUNCTION PROD.RAW.billing_account_id()
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'billing_account'
EXTERNAL_ACCESS_INTEGRATIONS = (microsoft_api_integration)
SECRETS = ('billing_account'=PROD.RAW.billing_account_id)
AS '
import _snowflake

def billing_account():   
  my_api_key = _snowflake.get_generic_secret_string(''billing_account'') 
  return my_api_key
';

In [ ]:

CREATE OR REPLACE FUNCTION PROD.RAW.mabl_api_key()
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'mabl_api_key'
EXTERNAL_ACCESS_INTEGRATIONS = (mabl_api_integration)
SECRETS = ('mabl_api_key'=PROD.RAW.mabl_api_key)
AS '
import _snowflake

def mabl_api_key():   
  my_api_key = _snowflake.get_generic_secret_string(''mabl_api_key'') 
  return my_api_key
';